In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
#import torchvision.transforms as transforms
import pandas as pd
import ast
torch.manual_seed(1234)

# Data Preprocessing to include label and shift it one place up

In [4]:
#Preprocessing
nepse_data = pd.read_csv("data_set_ready_for_training.csv")
nepse_data["news"] = nepse_data["news"].apply(ast.literal_eval)
#nepse_data.set_index("date", inplace=True)

In [5]:
nepse_data = nepse_data.replace([np.inf, -np.inf], np.nan).dropna()
nepse_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1318 entries, 0 to 1377
Data columns (total 11 columns):
closing_price    1318 non-null float64
news             1318 non-null object
momentum         1318 non-null float64
EMA              1318 non-null float64
MACD             1318 non-null float64
RSI              1318 non-null float64
ROI              1318 non-null float64
ATR              1318 non-null float64
williams         1318 non-null float64
CCI              1318 non-null float64
UO               1318 non-null float64
dtypes: float64(10), object(1)
memory usage: 123.6+ KB


In [7]:
#nepse_data['Label'] = nepse_data['closing_price'].rolling(window=2).apply(lambda x:  1 if x[1]>x[0] else 0 )
nepse_data['Label'] = nepse_data['closing_price'].shift(-1)
#nepse_data.Label = nepse_data.Label.shift(-1)
nepse_data = nepse_data[:-1]
nepse_data.head()

,closing_price,news,momentum,EMA,MACD,RSI,ROI,ATR,williams,CCI,UO,Label
0,975.0,"[[0.2306930273771286, 0.792356550693512, 0.642...",-87.0,1087.863864,-58.947313,0.149378,1.126154,106.0,0.820755,997.666667,0.154372,954.0
1,954.0,"[[0.31689509749412537, 0.8638337850570679, 0.6...",-2.0,1067.269422,-65.253876,0.250000,1.122642,38.0,1.000000,966.666667,0.105707,917.0
2,917.0,"[[0.31150949001312256, 0.7647785544395447, 0.6...",-75.0,1044.151049,-72.401798,0.480000,1.135878,75.0,1.000000,942.000000,0.069584,903.0
3,903.0,"[[0.1868913620710373, 0.7952941060066223, 0.61...",-87.0,1022.435502,-78.293327,0.000000,1.120598,87.0,1.000000,932.000000,0.068895,885.0
4,885.0,"[[0.3303453326225281, 0.8576347231864929, 0.68...",-90.0,1001.291578,-83.452392,0.000000,1.108927,90.0,1.000000,915.000000,0.108076,897.0


In [23]:
test =  nepse_data.loc[:,["closing_price", "momentum","EMA","MACD","RSI","ROI","ATR","williams",
                                "CCI","UO"]]
normalizer = StandardScaler()
normalizer.fit(test.values)
x_norm = pd.DataFrame(normalizer.transform(test.values), index=test.index, columns=test.columns)
x_norm
x_norm["news"] = nepse_data["news"]
x_norm["Label"] = nepse_data["Label"]
x_norm.head()

,closing_price,momentum,EMA,MACD,RSI,ROI,ATR,williams,CCI,UO,news,Label
0,-1.769629,-0.751830,-1.547293,-1.100855,-0.331177,2.667207,0.160012,0.727474,-1.725909,-1.280679,"[[0.2306930273771286, 0.792356550693512, 0.642...",954.0
1,-1.816039,0.010508,-1.593452,-1.219465,-0.319227,2.591076,-0.296228,1.158930,-1.794748,-1.462921,"[[0.31689509749412537, 0.8638337850570679, 0.6...",917.0
2,-1.897810,-0.644206,-1.645269,-1.353898,-0.291911,2.877979,-0.047980,1.158930,-1.849524,-1.598194,"[[0.31150949001312256, 0.7647785544395447, 0.6...",903.0
3,-1.928751,-0.751830,-1.693941,-1.464702,-0.348918,2.546782,0.032533,1.158930,-1.871730,-1.600776,"[[0.1868913620710373, 0.7952941060066223, 0.61...",885.0
4,-1.968531,-0.778736,-1.741331,-1.561730,-0.348918,2.293798,0.052661,1.158930,-1.909481,-1.454049,"[[0.3303453326225281, 0.8576347231864929, 0.68...",897.0


# From here on, data is prepared to be fed to pytorch network

The dataset is prepared by inheriting Dataset class, and DataLoader class is used for batching. 


In [24]:
#DataLoader class
class NepseDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.features = dataset.loc[:, ["closing_price", "momentum","EMA","MACD","RSI","ROI","ATR","williams",
                                "CCI","UO"]].values
        self.label =  dataset['Label'].values
        self.news = dataset['news'].values
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        tech_indicators = self.features[idx,:]
        label = self.label[idx]
        news = self.news[idx]
        sample = {'news': torch.tensor(news),'tech_indicators': torch.tensor(tech_indicators)
                  ,'label': torch.tensor(label)}
        return sample

In [25]:
def append_zeros(news):
    padding_dims = 20 - len(news)
    for _ in range(padding_dims):
        news.append([0]*100)
    return news
def truncate(news):
    return news[0:20]

In [26]:
def const_dimens(news):  
    if len(news)<20:
        news = append_zeros(news)
    elif len(news)>20:
        news = truncate(news)
    
    return news

In [34]:
x_norm["news"] = x_norm["news"].apply(const_dimens)

test_data = NepseDataset(x_norm)
print(len(test_data))
test_data[0]['news'].shape

1317


torch.Size([20, 100])

In [35]:
data = DataLoader(test_data, batch_size=1)
for i, sample in enumerate(data):
    print(sample)
    break

{'news': tensor([[[ 0.2307,  0.7924,  0.6421,  ..., -0.2635,  0.2692, -0.0625],
         [ 0.4144,  0.8800,  0.7002,  ..., -0.3542,  0.0610, -0.1523],
         [ 0.4557,  0.9929,  0.7184,  ..., -0.3667,  0.0949, -0.2086],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]), 'tech_indicators': tensor([[-1.7696, -0.7518, -1.5473, -1.1009, -0.3312,  2.6672,  0.1600,  0.7275,
         -1.7259, -1.2807]], dtype=torch.float64), 'label': tensor([954.])}


In [36]:
a = torch.randn(3,4)
b = torch.randn(3,6)
torch.cat((a,b),dim=1).shape

torch.Size([3, 10])

# Our Neural Network
 The network consists of CNN which extracts features from news, the output of which is concatenated with the technical indicators and fed to lstm. The output is then fed to softmax to predict the rise or fall in nepse for the following day.

In [44]:

class StockNN(nn.Module):
    def __init__(self, filter_sizes, drop_prob, embedding_dim, length_of_features, n_hidden_lstm=256, n_layers=1):
        super(StockNN, self).__init__()
        self.n_layers = n_layers
        self.n_hidden_lstm = n_hidden_lstm
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = 1, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.lstm = nn.LSTM(length_of_features, n_hidden_lstm, n_layers, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(n_hidden_lstm)
        self.linear = nn.Linear(n_hidden_lstm, 1)
        
        
    def forward(self,x,hc):
        
        #x_cnn = x['news']
        #x_cnn = x_cnn.reshape(x_cnn.shape[0],1,x_cnn.shape[1], x_cnn.shape[2])
        #print(x_cnn.shape)
        #conved = [F.relu(conv(x_cnn)).squeeze(3) for conv in self.convs]
        #print(x.shape for x in list(conved))
        #pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #print(x.shape for x in list(pooled))
        #stock_features = torch.cat(pooled, dim = 1)
        #print(stock_features.shape)
        #print(x['tech_indicators'].shape)
        
        #stock_features = torch.cat([stock_features,x['tech_indicators'].float()], dim=1)
        stock_features = x['tech_indicators'].float()
        #print(stock_features.shape)
        out, (h,c) = self.lstm(stock_features.view(stock_features.shape[0],1,-1),hc)
        #out = self.batch_norm(out.view(out.shape[0],-1))
        hidden_2_risefall = self.linear(out.view(out.shape[0],-1))
        rise_fall = F.relu(hidden_2_risefall)
        
        return rise_fall, (h,c)
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        return (torch.zeros(self.n_layers, batch_size, self.n_hidden_lstm),
                torch.zeros(self.n_layers, batch_size, self.n_hidden_lstm))


# Train Validation Split

In [45]:

def train_validate_set(data,val_frac=0):
    m = data.shape[0]
    val_idx = int(m*(1-val_frac))
    test_data, val_data = NepseDataset(data.head(val_idx)), NepseDataset(data.tail(m - val_idx))
    return test_data, val_data

td, vd = train_validate_set(nepse_data,0.2) 
print(len(td), len(vd))

1053 264


# Training the StockNN


In [46]:
#Have to check this one
def train(NN, data, val_frac=0.2, max_epochs=10, batch_size=10, learning_rate=1e-3):
    last_loss = 100
    NN.train()
    opt = torch.optim.Adam(NN.parameters(),lr=learning_rate)
    
    criterion = nn.MSELoss()
    
    # create training and validation data
    train_data, validate_data = train_validate_set(data,val_frac)
    dataloader = DataLoader(train_data, batch_size=batch_size)
    for e in range(max_epochs):
        hc = NN.init_hidden(batch_size)
        
        for i_batch, batched_sample in enumerate(dataloader):
            if len(batched_sample['label']) < batch_size:
                continue
            xtrain = dict(news = batched_sample["news"],tech_indicators = batched_sample['tech_indicators'])
            out, hc= NN(xtrain,hc)
        
            loss = criterion(out,batched_sample['label'].float())
            
            opt.zero_grad()
            loss.backward(retain_graph=True)
            
            opt.step()
            
            if(i_batch % 10 ==0):
                print(e, i_batch,loss.item(),("badyo" if last_loss < loss.item() else "ghatyo"))
                last_loss = loss.item()


  

In [47]:
c = dict(x = [2,3,4], y= [2,3,4], z= [6,7,9])
c['x']

[2, 3, 4]

In [48]:
stock_net = StockNN([2,2,3,3,4,4], 0, 100, 10)

In [50]:

train(stock_net, nepse_data, max_epochs= 10, batch_size=1, learning_rate=1e-3)

0 0 810170.875 badyo
0 10 763015.875 ghatyo
0 20 774417.5 badyo
0 30 741498.1875 ghatyo
0 40 640337.5625 ghatyo
0 50 606032.6875 ghatyo
0 60 529407.75 ghatyo
0 70 572362.625 badyo
0 80 586112.3125 badyo
0 90 584407.25 ghatyo
0 100 624990.0625 badyo
0 110 600077.1875 ghatyo
0 120 642758.75 badyo
0 130 1003568.75 badyo
0 140 1897890.0 badyo
0 150 1543297.625 ghatyo
0 160 1301709.375 ghatyo
0 170 1253706.125 ghatyo
0 180 1426825.625 badyo
0 190 1924588.875 badyo
0 200 1855140.75 ghatyo
0 210 1848948.0 ghatyo
0 220 1780896.5 ghatyo
0 230 1804341.5 badyo
0 240 1115264.625 ghatyo
0 250 1982470.25 badyo
0 260 1860152.375 ghatyo
0 270 2101644.0 badyo
0 280 2453882.0 badyo
0 290 2250665.75 ghatyo
0 300 2270982.5 badyo
0 310 3058015.25 badyo
0 320 3255786.0 badyo
0 330 2856163.25 ghatyo
0 340 2936747.5 badyo
0 350 2548530.5 ghatyo
0 360 2693502.0 badyo
0 370 2669903.0 ghatyo
0 380 2591428.0 ghatyo
0 390 2652392.75 badyo
0 400 3067545.25 badyo
0 410 3222845.0 badyo
0 420 2989430.5 ghatyo
0 430 28

KeyboardInterrupt: 

In [93]:
stock_net.eval()
hc = stock_net.init_hidden(1)
train_data, validate_data = train_validate_set(nepse_data,0.1)
dataloader = DataLoader(train_data, batch_size=1)
milyo = 0
for train in dataloader:
    out, hc= stock_net(train,hc)
    print(out)
    if (train["label"].data.cpu().numpy() ==np.argmax(out.data.cpu().numpy())):
        milyo += 1

tensor([[0.2037]], grad_fn=<ReluBackward0>)
tensor([[0.1899]], grad_fn=<ReluBackward0>)
tensor([[0.2095]], grad_fn=<ReluBackward0>)
tensor([[0.2060]], grad_fn=<ReluBackward0>)
tensor([[0.2068]], grad_fn=<ReluBackward0>)
tensor([[0.2340]], grad_fn=<ReluBackward0>)
tensor([[0.2159]], grad_fn=<ReluBackward0>)
tensor([[0.2126]], grad_fn=<ReluBackward0>)
tensor([[0.1671]], grad_fn=<ReluBackward0>)
tensor([[0.1680]], grad_fn=<ReluBackward0>)
tensor([[0.1881]], grad_fn=<ReluBackward0>)
tensor([[0.1633]], grad_fn=<ReluBackward0>)
tensor([[0.1494]], grad_fn=<ReluBackward0>)
tensor([[0.1311]], grad_fn=<ReluBackward0>)
tensor([[0.1545]], grad_fn=<ReluBackward0>)
tensor([[0.1535]], grad_fn=<ReluBackward0>)
tensor([[0.1511]], grad_fn=<ReluBackward0>)
tensor([[0.1510]], grad_fn=<ReluBackward0>)
tensor([[0.1288]], grad_fn=<ReluBackward0>)
tensor([[0.1272]], grad_fn=<ReluBackward0>)
tensor([[0.1336]], grad_fn=<ReluBackward0>)
tensor([[0.1487]], grad_fn=<ReluBackward0>)
tensor([[0.1466]], grad_fn=<Relu

tensor([[0.0877]], grad_fn=<ReluBackward0>)
tensor([[0.0881]], grad_fn=<ReluBackward0>)
tensor([[0.0877]], grad_fn=<ReluBackward0>)
tensor([[0.1075]], grad_fn=<ReluBackward0>)
tensor([[0.0938]], grad_fn=<ReluBackward0>)
tensor([[0.0763]], grad_fn=<ReluBackward0>)
tensor([[0.1134]], grad_fn=<ReluBackward0>)
tensor([[0.1240]], grad_fn=<ReluBackward0>)
tensor([[0.1161]], grad_fn=<ReluBackward0>)
tensor([[0.1147]], grad_fn=<ReluBackward0>)
tensor([[0.1061]], grad_fn=<ReluBackward0>)
tensor([[0.0866]], grad_fn=<ReluBackward0>)
tensor([[0.1066]], grad_fn=<ReluBackward0>)
tensor([[0.1171]], grad_fn=<ReluBackward0>)
tensor([[0.1168]], grad_fn=<ReluBackward0>)
tensor([[0.1191]], grad_fn=<ReluBackward0>)
tensor([[0.1176]], grad_fn=<ReluBackward0>)
tensor([[0.1111]], grad_fn=<ReluBackward0>)
tensor([[0.1173]], grad_fn=<ReluBackward0>)
tensor([[0.1135]], grad_fn=<ReluBackward0>)
tensor([[0.1145]], grad_fn=<ReluBackward0>)
tensor([[0.1156]], grad_fn=<ReluBackward0>)
tensor([[0.1223]], grad_fn=<Relu

tensor([[0.1592]], grad_fn=<ReluBackward0>)
tensor([[0.1589]], grad_fn=<ReluBackward0>)
tensor([[0.1583]], grad_fn=<ReluBackward0>)
tensor([[0.1587]], grad_fn=<ReluBackward0>)
tensor([[0.1594]], grad_fn=<ReluBackward0>)
tensor([[0.1586]], grad_fn=<ReluBackward0>)
tensor([[0.1593]], grad_fn=<ReluBackward0>)
tensor([[0.1592]], grad_fn=<ReluBackward0>)
tensor([[0.1590]], grad_fn=<ReluBackward0>)
tensor([[0.1321]], grad_fn=<ReluBackward0>)
tensor([[0.1611]], grad_fn=<ReluBackward0>)
tensor([[0.1609]], grad_fn=<ReluBackward0>)
tensor([[0.1565]], grad_fn=<ReluBackward0>)
tensor([[0.1568]], grad_fn=<ReluBackward0>)
tensor([[0.1567]], grad_fn=<ReluBackward0>)
tensor([[0.1568]], grad_fn=<ReluBackward0>)
tensor([[0.1560]], grad_fn=<ReluBackward0>)
tensor([[0.1520]], grad_fn=<ReluBackward0>)
tensor([[0.1283]], grad_fn=<ReluBackward0>)
tensor([[0.1335]], grad_fn=<ReluBackward0>)
tensor([[0.1411]], grad_fn=<ReluBackward0>)
tensor([[0.1434]], grad_fn=<ReluBackward0>)
tensor([[0.1534]], grad_fn=<Relu

tensor([[0.1326]], grad_fn=<ReluBackward0>)
tensor([[0.1314]], grad_fn=<ReluBackward0>)
tensor([[0.1443]], grad_fn=<ReluBackward0>)
tensor([[0.1365]], grad_fn=<ReluBackward0>)
tensor([[0.1383]], grad_fn=<ReluBackward0>)
tensor([[0.1151]], grad_fn=<ReluBackward0>)
tensor([[0.1405]], grad_fn=<ReluBackward0>)
tensor([[0.1432]], grad_fn=<ReluBackward0>)
tensor([[0.1304]], grad_fn=<ReluBackward0>)
tensor([[0.1469]], grad_fn=<ReluBackward0>)
tensor([[0.1581]], grad_fn=<ReluBackward0>)
tensor([[0.1363]], grad_fn=<ReluBackward0>)
tensor([[0.1570]], grad_fn=<ReluBackward0>)
tensor([[0.1431]], grad_fn=<ReluBackward0>)
tensor([[0.1494]], grad_fn=<ReluBackward0>)
tensor([[0.1599]], grad_fn=<ReluBackward0>)
tensor([[0.1457]], grad_fn=<ReluBackward0>)
tensor([[0.1334]], grad_fn=<ReluBackward0>)
tensor([[0.1272]], grad_fn=<ReluBackward0>)
tensor([[0.1387]], grad_fn=<ReluBackward0>)
tensor([[0.1369]], grad_fn=<ReluBackward0>)
tensor([[0.1267]], grad_fn=<ReluBackward0>)
tensor([[0.1092]], grad_fn=<Relu

tensor([[0.1912]], grad_fn=<ReluBackward0>)
tensor([[0.1907]], grad_fn=<ReluBackward0>)
tensor([[0.1928]], grad_fn=<ReluBackward0>)
tensor([[0.1999]], grad_fn=<ReluBackward0>)
tensor([[0.1923]], grad_fn=<ReluBackward0>)
tensor([[0.1925]], grad_fn=<ReluBackward0>)
tensor([[0.1913]], grad_fn=<ReluBackward0>)
tensor([[0.1796]], grad_fn=<ReluBackward0>)
tensor([[0.1639]], grad_fn=<ReluBackward0>)
tensor([[0.1671]], grad_fn=<ReluBackward0>)
tensor([[0.1696]], grad_fn=<ReluBackward0>)
tensor([[0.1438]], grad_fn=<ReluBackward0>)
tensor([[0.1327]], grad_fn=<ReluBackward0>)
tensor([[0.1321]], grad_fn=<ReluBackward0>)
tensor([[0.1436]], grad_fn=<ReluBackward0>)
tensor([[0.1583]], grad_fn=<ReluBackward0>)
tensor([[0.1620]], grad_fn=<ReluBackward0>)
tensor([[0.1642]], grad_fn=<ReluBackward0>)
tensor([[0.1525]], grad_fn=<ReluBackward0>)
tensor([[0.1586]], grad_fn=<ReluBackward0>)
tensor([[0.1341]], grad_fn=<ReluBackward0>)
tensor([[0.1382]], grad_fn=<ReluBackward0>)
tensor([[0.1375]], grad_fn=<Relu

tensor([[0.1393]], grad_fn=<ReluBackward0>)
tensor([[0.1449]], grad_fn=<ReluBackward0>)
tensor([[0.1557]], grad_fn=<ReluBackward0>)
tensor([[0.1384]], grad_fn=<ReluBackward0>)
tensor([[0.1360]], grad_fn=<ReluBackward0>)
tensor([[0.1471]], grad_fn=<ReluBackward0>)


In [57]:
print(milyo)

652
